In [ ]:
# Hangman

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [1]:
!pip install tensorflow==2.8.0
# !pip install gym
# !pip install keras
!pip install keras-rl2==1.0.4


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 497.6/497.6 MB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 40.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.5/462.5 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 44.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 59.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 781.3/781.3 kB 43.5 MB/s eta 0:00:00
  Attempting uninstall: keras
    Found existing installation: keras 2.15.0
    Uninstalling keras-2.15.0:
      Successfully uninstalled keras-2.15.0
  Attempting uninstall: tensorboard-data-server
    Found existing installation: tensorboard-data-server 0.7.2
    Uninstalling tensorboard-data-server-0.7.2:
      Successfully uninstalled tensorboard-data-server-0.7.2
  Attempting uninstall: google-auth-oauthlib
    Found existing 

In [2]:
from gym import Env
from gym.spaces import MultiBinary, Discrete
import numpy as np
import random


# Model things.
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Reshape, Masking
from tensorflow.keras.optimizers import Adam


# ImportError: cannot import name '__version__' from 'tensorflow.keras' (.local/lib/python3.8/site-packages/keras/api/_v2/keras/__init__.py)
# To solve the above error for my local system. adding the following lines.
import tensorflow as tf
from keras import __version__
tf.keras.__version__ = __version__

# Agent things.
from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory



/usr/local/lib/python3.10/dist-packages/tensorflow/python/framework/dtypes.py:205: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  np.bool8: (False, True),
/usr/local/lib/python3.10/dist-packages/tensorboard/compat/tensorflow_stub/dtypes.py:326: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  np.bool8: (False, True),


In [3]:
tf.__version__

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


'2.8.0'

In [12]:

class HangmanEnv(Env):
    """
    Class which acts as a Hangman environment where the learner can interact with.
    """

    def __init__(self, word: str) -> None:
        """
        Initialize the Hangman environment.
        """
        self.actual_word: str = word.lower()
        self.number_of_letters: int = len(self.actual_word)
        self.encoded_word: list[int] = [ord(letter) - 97 for letter in self.actual_word] # ord('a') = 97. I want 0 - a , 1 - b etc.
        self.letters_already_guessed: set[int] = set()
        self.gussed_prerior = set()
        self.guess = []
        self.letters_left_to_find = len(self.actual_word)
        # self.
        # Track the game, is done or not.
        self.guesses_left: int = 6
        # Actions we can take -- Chose any letter from a - z.
        self.action_space = Discrete(26)
        # Observation space --
        self.observation_space = MultiBinary((30, 27))
        # Set the starting state -- the starting state of the Hangman environment. (The initial input)
        self.state = self.create_state()

    def create_state(self):
        """
        Creates the initial state, the first input to out model.
        Should be a 3D array, since we are gonna use the LSTM model.
        * (n + 1, 27): n + 1 -- number of letters in the word + one rows extra which represents the letters already guessed.
        """
        encoded_word = self.encode_word() # (n, 27)
        encoded_already_guessed_letters = self.encode_already_guessed_letters() # (27,)
        word = np.append(encoded_word, [encoded_already_guessed_letters], axis=0)
        # Create the padding rows.
        # Create a 40 - (n+1) rows numpy array filled with '-1'.
        balance_row = 40 - (len(self.actual_word) + 1)
        pading_row = np.full((balance_row, 27), -1)

        return np.append(word, pading_row, axis=0)

    def encode_already_guessed_letters(self):
        """

        Encode already guessed letters.
        Create a 1 x 27 vector where 1s indicate that the letter was previously guessed.
        """
        guess = np.zeros(27, dtype=np.float32)
        for i in self.letters_already_guessed:
            guess[i] = 1
        return guess

    def encode_word(self):
        """
        Encode the original word into n x 27 numpy array. [n - number of letters in the original word,
        26 represents the number of letters and 27th represents it's a blank space.]
        """
        word: list[int] = [ascii_letter if ascii_letter in self.letters_already_guessed else 26 for ascii_letter in self.encoded_word]
        encoded_word = np.zeros((len(word), 27), dtype=np.float32)
        for i, j in enumerate(word):
            encoded_word[i, j] = 1
        return encoded_word

    def step(self, action):
        """
        Apply the action and get the reward for it.

        :param action: The action to take. Here deciding the chosen letter's value.
                       If the choose word is in the original word, encode it and change the state.
                       If not, keep the state unchanged and decrease the guesses_left.
                       Edit the guessed word in both the cases.
        """
        # print("action taken {}".format(action))
        guessed_letter = action

        # Store the letters guessed by the model.
        self.guess.append(guessed_letter)
        self.letters_already_guessed.add(guessed_letter)

        # Step 1: Decide the action value.
        # Case 1: If the guessed word is in original word.
        #         Encode and change the state.
        # Case 2: If the guessed word is not in original word.
        #         Change the state and decrease the guesses_left.

        self.state = self.create_state()

        #print(f"letters already gussed {self.letters_already_guessed}")
        if not guessed_letter in self.encoded_word and guessed_letter not in self.gussed_prerior:
            self.guesses_left -= 1

        # Step 2: Get the corresponding reward.
        # Case 1: If the guessed word is present in our original word and if it's not guessed before means, give the possitive reward.
        #         The reward will be added is, number of times the correct present in the actual word.

        # Case 2: If the guessed word not present in the actual word means, give the negative reward. (-1)

        if guessed_letter in self.encoded_word and guessed_letter not in self.gussed_prerior:

            number_of_correct_rewards = self.encoded_word.count(guessed_letter)
            self.letters_left_to_find -= number_of_correct_rewards
            #print("letters left to find {}".format(self.letters_left_to_find))
            reward: int = number_of_correct_rewards
        else:
            reward: int = -1

        # Store the guessed letter, this will help as to determine the rewards.
        self.gussed_prerior.add(guessed_letter)


        # Step 3: Deicide the game is done or not.
        if self.guesses_left == 0 or self.letters_left_to_find <= 0:
            done: bool = True
        else:
            done: bool = False

        # Step 4: Generate the info message.
        # We can just put the placeholder for it, for now.
        info: dict = {}

        return self.state, reward, done, info

    def render(self):
        """
        This function is used to display the game, nothing to show for now leaving it empty.
        """
        pass

    def reset(self):
        """
        Reset the game environment to the beginning state and return the beginning state.
        """
        # print("Called teh reset method ")
        #print("**"*50)
        self.guess = []
        self.gussed_prerior = set()
        self.letters_left_to_find = len(self.actual_word)
        self.letters_already_guessed = set()
        self.state = self.create_state()
        self.guesses_left = 6

        return self.state


In [13]:
env = HangmanEnv('appartment')
env.action_space.n


done = False
score = 0
while not done:
    #action = int(input())
    action = env.action_space.sample()
    observation, reward, done, info = env.step(action)
    score += reward
    print(f"The total score{score}")
    print("--"*50)

env.close()


The total score1
----------------------------------------------------------------------------------------------------
The total score0
----------------------------------------------------------------------------------------------------
The total score1
----------------------------------------------------------------------------------------------------
The total score2
----------------------------------------------------------------------------------------------------
The total score1
----------------------------------------------------------------------------------------------------
The total score0
----------------------------------------------------------------------------------------------------
The total score-1
----------------------------------------------------------------------------------------------------
The total score-2
----------------------------------------------------------------------------------------------------
The total score-3
------------------------------------

In [ ]:
# Get the input from the text file.
file_path = "/content/drive/MyDrive/Colab Notebooks/Hangman/words_250000_train.txt"
words_list: list = []
w:dict = {}
with open(file_path, 'r') as file:
    for line in file:
        # Trim the words and remove the words that are not only alphabets.
        word = line.strip()
        if len(word) > 0 and word.isalpha() and word not in w:
            words_list.append(word)
            w[word] = 1
len(words_list)

227300

In [ ]:
# Clean the words file, select only correct spelling words.


# from nltk.corpus import words
# import nltk
# nltk.download('words')
# english_words = words.words()
# w = []
# #word_checker = enchant.Dict("en_US")
# for word in words_list:
#     if  word in english_words:
#       w.append(word)


In [ ]:
# Store the Words in a pickle file for later use.


# import pickle
# with open("my_file.pickle", "wb") as f:
#   pickle.dump(w, f)

In [ ]:
# Load the words from the pickle file.
import pickle
with open("/content/my_file.pickle", "rb") as f:
    words = pickle.load(f)


In [ ]:
train_val_split_idx = int((len(words)) * 0.8)
print('Training with {} words...'.format(train_val_split_idx))

max_num_of_letters = max([len(i) for i in words[:train_val_split_idx]])
epoch_size = train_val_split_idx
Epoches_count = 5
batch_size = np.array([len(i) for i in words[:train_val_split_idx]]).mean()
print("Max word length: {}!".format(max_num_of_letters))


# Split train and test data
print("Splitting train and test data...")
train , test = words[:train_val_split_idx], words[train_val_split_idx:]
train_len = len(train)
test_len = len(test)
print(f"Train data length -- {train_len}")
print(f"Train data length -- {test_len}")

# The number of actions a agent can take. here guessing a letter from a - z.
actions = 26

Training with 84363 words...
Max word length: 24!
Splitting train and test data...
Train data length -- 84363
Train data length -- 21091


In [ ]:
# Create the model.
def create_model(state_shape, action_shape):
    """
    Build the sequential model.

    :param state_shape: The shape of the state. (shape of the input)
    :param action_shape: The shape of the action. (shape of the output)
    """
    model = Sequential()

    # Reshaing because the keral-rl adds one extra dimentions to the input. Which will be a 4d array in the shape of (1,1, n_timestamp, n_features).
    # But LSTM Accepts the input as 3d array. (n_batch, n_timestamp, n_features).
    model.add(Reshape((40, 27), input_shape=(1, 40, 27)))

    # Add the masking layer. -- masking value i can kept is -1.
    model.add(Masking(mask_value=-1))

    # add one layer of lstm and dropout regularization.
    model.add(LSTM(27, return_sequences=True, input_shape=(None,None,27), dropout=0.2))

    # add one layer of lstm and dropout regularization.
    model.add(LSTM(27, dropout=0.2))

    # output layer.
    model.add(Dense(action_shape, activation='linear')) # pick a letter out of 26.

    return model
model = create_model(0, actions)
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 reshape_1 (Reshape)         (None, 40, 27)            0         
                                                                 
 masking_1 (Masking)         (None, 40, 27)            0         
                                                                 
 lstm_2 (LSTM)               (None, 40, 27)            5940      
                                                                 
 lstm_3 (LSTM)               (None, 27)                5940      
                                                                 
 dense_2 (Dense)             (None, 26)                728       
                                                                 
Total params: 12,608
Trainable params: 12,608
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# tf.compat.v1.disable_eager_execution()
# tf.compat.v1.experimental.output_all_intermediates(True)
tf.compat.v1.disable_eager_execution()


In [ ]:
# Not possible to train with all the words, since the model taking so long time to train.
# So making a group of 26 having starting letter same and taking 100 from each.
# d = {'a': [],'b': [],'c': [],'d': [],'e': [],'f': [],'g':[],'h':[],'i':[],'j':[],'k':[],
#      'l':[],'m':[],'n':[],'o':[],'p':[],'q':[],'r':[],'s':[],'t':[],'u':[],'v':[],'w':[],'x':[],'y':[],'z':[]}

# for w in words:
#   for alpahet in d.keys():
#     if w.startswith(alpahet):
#       d[alpahet].append(w)
#       break
# d
# import pickle
# with open("words_ordered.pickle", "wb") as f:
#   pickle.dump(d, f)
# with open("/content/words_ordered.pickle", "rb") as f:
#     words_ordered = pickle.load(f)
# words_trainable = []
# for k in words_ordered.keys():
#   words_trainable.extend(words_ordered[k][:100])
# len(words_trainable)
# import pickle
# with open("words_choosen.pickle", "wb") as f:
#   pickle.dump(words_trainable, f)

with open("/content/words_choosen.pickle", "rb") as f:
    words = pickle.load(f)

In [ ]:
import random
print("Splitting train and test data...")
random.shuffle(words)
train_val_split_idx = int((len(words)) * 0.8)
train , test = words[:train_val_split_idx], words[train_val_split_idx:]
train_len = len(train)
test_len = len(test)
print(f"Train data length -- {train_len}")
print(f"Train data length -- {test_len}")

Splitting train and test data...
Train data length -- 2080
Train data length -- 520


In [ ]:
# Build the agent with keras rl.
import keras

def build_agent(model, actions):
    policy = BoltzmannQPolicy()
    memory = SequentialMemory(limit=50000, window_length=1)
    dqn = DQNAgent(model=model, memory=memory, policy=policy,
                  nb_actions=actions, nb_steps_warmup=100, target_model_update=1e-2)
    return dqn

dqn = build_agent(model, actions)

dqn.compile(tf.keras.optimizers.Adam(learning_rate=1e-3),metrics=[keras.metrics.CategoricalCrossentropy()])



# NUM_EPOCHS = 10
# for epoch in range(NUM_EPOCHS):

#   dqn.fit(enviroment, nb_steps=1000, visualize=False, verbose=1)
# total_samples = 0

train = train[10:1000]
# the actual training happening here.
for epoch in range(Epoches_count):
    for i in range(len(train)):
        word = train[i]
        print(">>"*50)
        print("EPOCH {}".format(epoch))
        print(">>"*50)
        print("THe word which is training is ....{}, index {}".format(word, i))
        print(">>"*50)


        enviroment = HangmanEnv(word = word)

        dqn.fit(enviroment, nb_steps=1000, visualize=False, verbose=1)
#   dqn.fit(enviroment, nb_steps=1000, visualize=False, verbose=1)
# total_samples = 0

train = train[15:1000]
# the actual training happening here.
for epoch in range(Epoches_count):
    for i in range(len(train)):
        word = train[i]
        print(">>"*50)
        print("EPOCH {}".format(epoch))
        print(">>"*50)

    file_name = f'/content/drive/MyDrive/Colab Notebooks/Hangman/dqn_weights{epoch}.h5f'
    dqn.save_weights(file_name, overwrite=True)

In [ ]:
dqn.save_weights('dqn_weights.h5f', overwrite=True)


In [ ]:
e = HangmanEnv("aachen")
scores = dqn.test(e, nb_episodes=10, visualize=False)
print(np.mean(scores.history['episode_reward']))


Testing for 10 episodes ...
Episode 1: reward: 6.000, steps: 5
Episode 2: reward: 6.000, steps: 5
Episode 3: reward: 6.000, steps: 5
Episode 4: reward: 6.000, steps: 5
Episode 5: reward: 6.000, steps: 5
Episode 6: reward: 6.000, steps: 5
Episode 7: reward: 6.000, steps: 5
Episode 8: reward: 6.000, steps: 5
Episode 9: reward: 6.000, steps: 5
Episode 10: reward: 6.000, steps: 5
6.0


In [ ]:
# dqn.save_weights('dqn_weights.h5f', overwrite=True)
# del model
# del dqn
# del env

In [ ]:
# env = gym.make('CartPole-v0')
# actions = env.action_space.n
# states = env.observation_space.shape[0]
# model = build_model(states, actions)
# dqn = build_agent(model, actions)
# dqn.compile(Adam(lr=1e-3), metrics=[])
# dqn.load_weights('dqn_weights.h5f')
# _ = dqn.test(env, nb_episodes=5, visualize=True)
